<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GPT4_AI_FLIGHT_PLAN_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython import get_ipython
from IPython.display import display

In [2]:
!nvidia-smi

Thu Jan  9 03:45:27 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   44C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# Install required libraries
!pip install --upgrade datasets accelerate evaluate --quiet
!pip install trl --quiet
!pip install openai --quiet
!pip install requests --quiet # install requests
!pip install python-dotenv --quiet # install python-dotenv

# Install colab-env
!pip install colab-env -q

!pip install faiss-gpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 25.0 MB/s eta 0:00:00


In [8]:
import os
from dotenv import load_dotenv
import random
import time

import faiss
import folium
import numpy as np
from geopy.geocoders import Nominatim
from openai import OpenAI

import colab_env  # Import colab_env

!rm -rf /content/flight_path.html

# Load the .env file
load_dotenv()

# Set your OpenAI API key
openai_api_key = os.environ.get("OPENAI_API_KEY")
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set.")

# Initialize the OpenAI client
client = OpenAI(api_key=openai_api_key)

# Model ID for GPT-4
model_id = "gpt-4"

# Initialize Faiss index
embedding_size = 1536  # Adjust based on the model's output
index = faiss.IndexFlatL2(embedding_size)

# Cache for storing generated flight plans and their embeddings
flight_plan_cache = {}


def get_embedding(text):
    """Gets the embedding of the text using the OpenAI API."""
    response = client.embeddings.create(input=text,
                                       model="text-embedding-ada-002")
    embedding = response.data[0].embedding
    return np.array(embedding, dtype=np.float32)


def generate_response(prompt, max_new_tokens=1024):
    """Generates a response using GPT-4, possibly using cached results."""

    # Calculate embedding for the prompt
    embedding = get_embedding(prompt)

    # Search for similar flight plans in the index
    D, I = index.search(embedding.reshape(1, -1), k=1)

    # Set a similarity threshold (adjust as needed)
    similarity_threshold = 0.2
    if D[0][0] < similarity_threshold:
        print("Using similar flight plan from memory...")
        similar_prompt = list(flight_plan_cache.keys())[I[0][0]]
        return flight_plan_cache[similar_prompt]

    print("Generating new flight plan...")

    messages = [{"role": "user", "content": prompt}]

    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=messages,
            max_tokens=max_new_tokens,
            temperature=0.7,
        )
    except Exception as e:
        return f"An error occurred: {e}"

    if response is None or not response.choices:
        response_text = "The model could not generate a response for your prompt."
    else:
        response_text = response.choices[0].message.content

    flight_plan_cache[prompt] = response_text
    index.add(embedding.reshape(1, -1))
    return response_text


def get_simulated_weather(location):
    """Simulates weather data for a given location."""
    weather_conditions = ["Clear", "Cloudy", "Rainy", "Snowy", "Foggy"]
    temperature_ranges = {
        "New York": (-5, 30),
        "New York City": (-5, 30),
        "new york city": (-5, 30),
        "Shanghai": (0, 35),
        "shanghai": (0, 35),
    }
    wind_speed_range = (0, 20)
    humidity_range = (30, 100)

    condition = random.choice(weather_conditions)
    temperature_min, temperature_max = temperature_ranges.get(location, (0, 25))
    temperature = round(random.uniform(temperature_min, temperature_max), 1)
    feels_like = round(temperature + random.uniform(-2, 2), 1)
    wind_speed = round(random.uniform(*wind_speed_range), 1)
    humidity = random.randint(*humidity_range)

    weather_info = (
        f"Simulated weather in {location}:\n"
        f"Temperature: {temperature}°C (feels like {feels_like}°C)\n"
        f"Conditions: {condition}\n"
        f"Humidity: {humidity}%\n"
        f"Wind speed: {wind_speed} m/s")

    return weather_info


def create_flight_plan(
        origin,
        destination,
        aircraft_type,
        flight_date,
        preferred_route=None,
        cruising_altitude=None,
        departure_time=None,
):
    """Creates a flight plan using GPT-4."""

    prompt = f"""
    ## Flight Plan Request:

    **Origin:** {origin}
    **Destination:** {destination}
    **Aircraft Type:** {aircraft_type}
    **Flight Date:** {flight_date}
    """

    if preferred_route:
        prompt += f"**Preferred Route:** {preferred_route}\n"
    if cruising_altitude:
        prompt += f"**Cruising Altitude:** {cruising_altitude}\n"
    if departure_time:
        prompt += f"**Departure Time:** {departure_time}\n"

    prompt += """

    **Please generate a detailed flight plan including:**

    * **Route:** with waypoints (if applicable)
    * **Estimated Flight Duration:**
    * **Fuel Requirements:** (estimated)
    * **Cruising Altitude:** (if not specified)
    * **Departure Time:** (if not specified)
    * **Estimated Arrival Time:**
    * **Alternate Airports:** (if applicable)
    * **Fuel Consumption:** (estimated) in kg/hour

    **Format the flight plan clearly for readability.**
    """

    flight_plan = generate_response(prompt)
    return flight_plan


# --- Agent framework ---


class FlightPlanAgent:

    def __init__(self):
        self.memory = {"previous_flight_plans": {}}

    def extract_flight_info(self, user_input):
        """
        Extracts origin and destination from user input, handling multi-word locations.
        """
        origin = None
        destination = None
        words = user_input.lower().split()

        if "from" in words and "to" in words:
            from_index = words.index("from")
            to_index = words.index("to")

            origin_words = []
            for i in range(from_index + 1, to_index):
                origin_words.append(words[i])
            origin = " ".join(origin_words)

            destination_words = words[to_index + 1:]
            destination = " ".join(destination_words)

        if origin is not None:
            origin = " ".join(word.capitalize() for word in origin.split())
            origin = origin.strip()
        if destination is not None:
            destination = " ".join(word.capitalize() for word in destination.split())
            destination = destination.strip()

        return origin, destination

    def get_weather(self, location):
        """
        This now simulates weather information.
        """
        return get_simulated_weather(location)

    def interact(self, user_input):
        origin, destination = self.extract_flight_info(user_input)

        if origin and destination:
            key = f"{origin}-{destination}"
            if key in self.memory["previous_flight_plans"]:
                previous_plan = self.memory["previous_flight_plans"][key]
                return (
                    f"I found a previous flight plan from {origin} to {destination}: \n\n"
                    f"{previous_plan}\n\n"
                    "Would you like to create a new one?")

            origin_weather = self.get_weather(origin)
            destination_weather = self.get_weather(destination)

            flight_plan = create_flight_plan(
                origin,
                destination,
                "Boeing 777",
                "2024-01-15",
            )
            self.memory["previous_flight_plans"][key] = flight_plan

            # --- Visualization code ---
            geolocator = Nominatim(user_agent="flight_plan_app")

            def get_coordinates(location):
                """Gets the latitude and longitude coordinates for a location with retry."""
                for attempt in range(3):  # Retry up to 3 times
                    try:
                        # Use more specific query for airports
                        if "airport" in location.lower():
                            location_data = geolocator.geocode(
                                location,
                                timeout=10)  # Increased timeout to 10 seconds
                        else:
                            location_data = geolocator.geocode(
                                location,
                                timeout=10)  # Increased timeout to 10 seconds
                        if location_data:
                            print(
                                f"Coordinates found for {location}: {location_data.latitude}, {location_data.longitude}"
                            )
                            return (location_data.latitude, location_data.longitude)
                        else:
                            print(f"Location not found: {location}")
                            return None
                    except Exception as e:
                        print(
                            f"Error getting coordinates for {location} (attempt {attempt + 1}): {e}"
                        )
                        time.sleep(2)  # Wait for 2 seconds before retrying
                return None  # Return None if all attempts fail

            m = folium.Map(location=[50, -30], zoom_start=3)

            # Extract waypoints from the generated flight plan
            # (this might need adjustment based on the format of the flight plan)
            waypoints_from_plan = []
            for line in flight_plan.split("\n"):
                if "-" in line and origin in line and destination in line:
                    waypoints_from_plan = [
                        wp.strip() for wp in line.split("-")[1:-1]
                    ]
                    break

            waypoints = [
                get_coordinates(origin),
                *[
                    get_coordinates(waypoint.strip())
                    for waypoint in waypoints_from_plan
                ],
                get_coordinates(destination),
            ]

            waypoints = [waypoint for waypoint in waypoints if waypoint is not None]

            if len(waypoints) >= 2:
                folium.PolyLine(waypoints,
                                color="blue",
                                weight=2.5,
                                opacity=1).add_to(m)
            else:
                print(
                    "Not enough valid waypoints to draw the flight path on the map."
                )

            folium.Marker(location=waypoints[0],
                          popup=origin,
                          tooltip="Origin").add_to(m)
            folium.Marker(location=waypoints[-1],
                          popup=destination,
                          tooltip="Destination").add_to(m)

            m.save("flight_path.html")
            # --- End of visualization code ---

            return (
                f"Here's a flight plan from {origin} to {destination}: \n\n"
                f"{flight_plan}\n\n"
                f"Weather in {origin}: {origin_weather}\n"
                f"Weather in {destination}: {destination_weather}\n\n"
                f"The flight path has been visualized and saved as 'flight_path.html'"
            )
        else:
            return "I can create flight plans. Please specify origin and destination (e.g., 'flight plan from New York to Shanghai')"


# flight path from new york city to shanghai

# Create the agent
agent = FlightPlanAgent()

# Interaction loop
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    response = agent.interact(user_input)
    print(f"Agent: {response}")

Generating new flight plan...
Coordinates found for New York City: 40.7127281, -74.0060152
Coordinates found for Shanghai: 31.2312707, 121.4700152
Agent: Here's a flight plan from New York City to Shanghai: 

## Flight Plan:

**Flight Details:**
- **Origin:** New York City (JFK)
- **Destination:** Shanghai (PVG)
- **Aircraft Type:** Boeing 777
- **Flight Date:** 2024-01-15

**The Flight Route:**
- **Departure:** New York City - JFK
- Waypoint 1: Boston - BOS
- Waypoint 2: Toronto - YYZ
- Waypoint 3: Thunder Bay - YQT
- Waypoint 4: Edmonton - YEG
- Waypoint 5: Anchorage - ANC
- Waypoint 6: Petropavlovsk-Kamchatsky - PKC
- Waypoint 7: Khabarovsk - KHV
- **Arrival:** Shanghai - PVG

**Estimated Flight Duration:**
- The estimated total flying time is approximately 15 hours 30 minutes.

**Fuel Requirements:**
- The estimated total fuel requirement for this flight is approximately 150,000 kg.

**Cruising Altitude:**
- The cruising altitude for this flight will be 35,000 feet.

**Departure Ti